# ReAct Agent Using Function Calling


<a href="https://colab.research.google.com/github/dair-ai/maven-pe-for-llms-7/blob/main/demos/session-4/react.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's first demo this with LangChain:

In [173]:
%%capture
!pip install langchain
!pip install langchain_openai
!pip install google-search-results
!pip install langchainhub
!pip install numexpr
!pip install metaphor-python

## LangChain

In [178]:
from langchain.agents import load_tools
from langchain.agents import tool
from langchain.agents import AgentType
from langchain.agents import initialize_agent
from langchain import hub
from langchain_core.messages import AIMessage, HumanMessage
from langchain.chains import LLMMathChain
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_community.tools.convert_to_openai import format_tool_to_openai_tool

from dotenv import load_dotenv
import openai
import os

# load the environment variables
load_dotenv()

# API configuration
openai.api_key = os.getenv("OPENAI_API_KEY")

# for LangChain
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# you need to obtain a SERP API key from https://serpapi.com/
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

In [10]:
def get_completion(messages, model="gpt-3.5-turbo-0613", temperature=0, max_tokens=300):
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message.content

In [11]:
prompt = """Which team won the 2023 NBA finals?"""

message = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(message)
print(response)

I'm sorry, but as an AI language model, I don't have access to real-time information or future events. The 2023 NBA Finals have not occurred yet, so I cannot provide you with the winner.


Now let's create a simple agent to look for information from a search API like `serpapi`.

We define the tools:

In [17]:
# define a prompt
prompt = hub.pull("hwchase17/react")

In [18]:
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

Define model and tools:

In [49]:
llm = OpenAI(temperature=0)

tools = load_tools(["serpapi"], llm=llm)

Create our agent:

In [50]:
agent = create_react_agent(llm, tools, prompt)

Run our agent:

In [51]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [52]:
agent_executor.invoke({"input": "Which team won the 2023 NBA finals"})



> Entering new AgentExecutor chain...
 I should always think about what to do
Action: Search
Action Input: 2023 NBA finals winnerDenver NuggetsI should double check the source and date of this information
Action: Search
Action Input: Denver Nuggets 2023 NBA finals{'title': 'Denver Nuggets', 'rankings': '1st in Western Conference', 'thumbnail': 'https://serpapi.com/searches/65b1f78949ecdbc4ebc777e3/images/e8c7227732e418350150b6995d8dcbc2aa88d42581c1374e931a04afd2b05b8e.png', 'games': [{'tournament': 'NBA', 'arena': 'Ball Arena', 'status': 'Final', 'date': 'Jun 12, 23', 'video_highlights': {'link': 'https://www.youtube.com/watch?v=ucZZdf94LbI&feature=onebox', 'thumbnail': 'https://serpapi.com/searches/65b1f78949ecdbc4ebc777e3/images/e8c7227732e41835cf98f69076e720e3f99417670a75443acccc4f27506aca4c6168ea75b877d17f9846cfc9ee9ae18e763801943e474394.jpeg', 'duration': '9:51'}, 'teams': [{'name': 'Heat', 'score': '89', 'thumbnail': 'https://serpapi.com/searches/65b1f78949ecdbc4ebc777e3/images

{'input': 'Which team won the 2023 NBA finals',
 'output': 'The Denver Nuggets won the 2023 NBA finals.'}

Another question:

In [48]:
agent_executor.invoke({"input": "Who is leading the NBA Western Conference?"})



> Entering new AgentExecutor chain...
 I should always think about what to do
Action: Search
Action Input: NBA Western Conference standings{'title': 'NBA standings', 'thumbnail': 'https://serpapi.com/searches/65b1f420e135086f455bd473/images/557b4a4265f9ece78704b4078edffe0754b56c436d9411b481518cd5f6b575d0.png', 'season': '2023–24', 'round': 'Regular season', 'league': {'standings': [{'team': {'thumbnail': 'https://serpapi.com/searches/65b1f420e135086f455bd473/images/557b4a4265f9ece79ea7b8c5eaaca2f5b4b64b1122b76d7cd05b5d45fce0442a17396e6382fb46d5a202d74275a94a6ba15b74ea58b338e3.png', 'name': 'Thunder'}, 'pos': '1', 'w': '31', 'l': '13', 'pct': '.705', 'gb': '-', 'conf': '19-10', 'home': '17-5', 'away': '14-8', 'l10': '8-2', 'strk': 'W4'}, {'team': {'thumbnail': 'https://serpapi.com/searches/65b1f420e135086f455bd473/images/557b4a4265f9ece79ea7b8c5eaaca2f5b4b64b1122b76d7cd05b5d45fce0442a33fc2aa2d612f5a084525f5573433ccfd2a6c27964c74bdf.png', 'name': 'Timberwolves'}, 'pos': '2', 'w': '31',

{'input': 'Who is leading the NBA Western Conference?',
 'output': 'The Oklahoma City Thunder are currently leading the NBA Western Conference with a record of 31 wins and 13 losses.'}

### ReAct Chat

In [135]:
# replicated below:

# chat_prompt = hub.pull("hwchase17/react-chat")

In [161]:
chat_prompt = PromptTemplate(
    input_variables=['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools'], template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.\n\nOverall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.\n\nTOOLS:\n------\n\nAssistant has access to the following tools:\n\n{tools}\n\nTo use a tool, please use the following format:\n\n```\nThought: Do I need to use a tool? Yes\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result after the action\n```Please make sure to use those indicators like Observation and Thoughts.\n\nWhen you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:\n\n```\nThought: Do I need to use a tool? No\nFinal Answer: [your response here]\n```\n\nBegin!\n\nPrevious conversation history:\n{chat_history}\n\nNew input: {input}\n{agent_scratchpad}'
)

In [162]:
# Construct the ReAct agent
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi"], llm=llm)

# executor
agent = create_react_agent(llm, tools, chat_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# invoke it
agent_executor.invoke(
    {
        "input": "Who is leading the NBA Western Conference?",
        # Notice that chat_history is a string, since this prompt is aimed at LLMs, not chat models
        "chat_history": "Not much found yet.",
    }
)



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Search
Action Input: Who is leading the NBA Western Conference?{'title': 'NBA standings', 'thumbnail': 'https://serpapi.com/searches/65b2048dc47d3ccb6089ff68/images/3048875d3f3a0812ffc691e3d7c3aa6351ebff48720b23eba0bbc3f2b36fec84.png', 'season': '2023–24', 'round': 'Regular season', 'league': {'standings': [{'team': {'thumbnail': 'https://serpapi.com/searches/65b2048dc47d3ccb6089ff68/images/3048875d3f3a0812a430546ffa07984cbdbf946e3285ccd590b12508666aa7c8b143454c15f4d86bf6002fc88fd51eee3792939b5a0e9a5f.png', 'name': 'Thunder'}, 'pos': '1', 'w': '31', 'l': '13', 'pct': '.705', 'gb': '-', 'conf': '19-10', 'home': '17-5', 'away': '14-8', 'l10': '8-2', 'strk': 'W4'}, {'team': {'thumbnail': 'https://serpapi.com/searches/65b2048dc47d3ccb6089ff68/images/3048875d3f3a0812a430546ffa07984cbdbf946e3285ccd590b12508666aa7c810d676627cb8259c3510044b8e8529d46cb4238527fe119a.png', 'name': 'Timberwolves'}, 'pos': '2', '

{'input': 'Who is leading the NBA Western Conference?',
 'chat_history': 'Not much found yet.',
 'output': 'The Oklahoma City Thunder are currently leading the NBA Western Conference.'}